In [1]:
import random

class Sentiment:
    Negative = 'Sorry for the disappointment'
    Positive = 'Thanks, It means alot'
    Netural = "Ok, we'll look into it"
class Review:
    def __init__(self,text,score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <=2:
            return Sentiment.Negative
        elif self.score >2 and self.score <=3:
            return Sentiment.Netural
        else:
            return Sentiment.Positive
#  We have break the values individually based on the sentiment and make it equal dataset for creating the model       
class ReviewContainer:
    def __init__(self,reviews):
        self.reviews = reviews
        
    def get_text(self):
# Splitting the text data from the reviews dataset        
        return [x.text for x in self.reviews]
    
    def get_comment(self):
# Splitting the sentiment data from the reviews dataset        
        return [x.sentiment for x in self.reviews]
# This one helps to split the values based on the sentiment
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.Negative,self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.Positive,self.reviews))
        Neutral = list(filter(lambda x: x.sentiment == Sentiment.Netural,self.reviews))
 
# we are making it equal number of positive values to negative

        positive_shrunk = positive[:len(negative)]
        self.reviews = negative+positive_shrunk
        random.shuffle(self.reviews)

In [16]:
len(reviews)

10000

### Data Preparation

In [4]:
import json

file_name = 'Books_small_10000.json'

In [5]:
reviews = []
with open(file_name) as f:
    for file in f:
        review = json.loads(file)
        reviews.append(Review(review['reviewText'],review['overall']))

In [6]:
print(reviews[4].score)
print(reviews[5].sentiment)

3.0
Thanks, It means alot


In [7]:
review

{'reviewerID': 'A1EZD7IJOOAF6S',
 'asin': '0956998569',
 'reviewerName': 'Amazon Customer',
 'helpful': [1, 1],
 'reviewText': "Highly recommend this entire trilogy. It is very well written and held me in suspense and kept me reading.  Even with the same old young girl heroine who goes head strong and hell bent on saving the new world, id tecommend this book to dystopian fiction fans!  Not overdone, thankfully! A fesw situations made it feel like I've read this same plot before....but these were well thought out and much better written!  This authr has a gift a d I will be looking forward to reading more of ber work.",
 'overall': 4.0,
 'summary': 'truly enjoyed',
 'unixReviewTime': 1402358400,
 'reviewTime': '06 10, 2014'}

### Model Creation

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
training,test = train_test_split(reviews,test_size=0.33,random_state=42)

# Here we are splitting the training and test data based on the ReviewContainer class
train_cont=ReviewContainer(training)
test_cont = ReviewContainer(test)


### Note

From the f1_score values still the negative prediction score is very low as compared to positive.
Which is due to high positive sentiments in the test_y dataset.
To improve the f1_score we are making the test data also evenly distributed.


In [33]:
train_cont.evenly_distribute()
train_x = train_cont.get_text()
train_y = train_cont.get_comment()

test_cont.evenly_distribute()
test_x = test_cont.get_text()
test_y = test_cont.get_comment()

In [34]:
print(train_y.count(Sentiment.Positive))
print(train_y.count(Sentiment.Negative))

436
436


In [35]:
print(test_y.count(Sentiment.Positive))
print(test_y.count(Sentiment.Negative))

208
208


In [36]:
print(train_x[0])
print(train_y[0])

print(test_x[4])
print(test_y[4])

I found this book to be very interesting. Started and didn't want to put it down. Enjoyed it very much. Good ending.
Thanks, It means alot
Recommended by a friend as &#34;one of the best books I have read.&#34;  I would not go that far but it is a page turner and very touching story.  Delightful read.
Thanks, It means alot


In [37]:
len(test_y)

416

In [87]:
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer #This another type of vectorizer

# vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer()
# Converting string to binary machine understanding
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)
# print(train_x_vectors[0].get_feature_names())
# It helps to get the list of words in the training data
print(len(vectorizer.get_feature_names()))
train_x_vectors[0]
test_x_vectors

8906


<416x8906 sparse matrix of type '<class 'numpy.float64'>'
	with 24092 stored elements in Compressed Sparse Row format>

In [70]:
train_x_vectors

<872x8906 sparse matrix of type '<class 'numpy.float64'>'
	with 53647 stored elements in Compressed Sparse Row format>

### SVM

In [71]:
test_x[98]

"LOVE THE SERIES OF BOOKS WITH HARRY HOLE. THEY ARE REALLY EXCITING AND A LITTLE BIT DARK. HE IS A GREAT AUTHOR. DON'T MISS ANY OF HIS BOOKS."

In [72]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors,train_y)

print(test_x[0])

clf_svm.predict(test_x_vectors[0])

I'm sure all the sanctioned words are in there, but how to find them while in game?  There is no search feature and who has time to scroll through the whole alphabet to check your hoped for word?  It takes so long!  Your friends will chuck you out of the game.  Oh, yes they will. Later, when you have no more friends to play with you can scroll and scroll and scroll to use up your now desolate life.  :P


array(['Thanks, It means alot'], dtype='<U28')

### Decision Tree

In [73]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors,train_y)

print(test_x[22])
clf_dec.predict(test_x_vectors[22])

This odd and pretentious novel is based on the true case of an innocent man who falsely confessed to a series of homicides. The nation is on edge in the wake of a series of mysterious disappearances. The targets, all older, solitary sorts, vanish and their presumed abductor leaves nary a clue but for a marked playing card. Oda Sotatsu is a young man living a life both unfulfilling and uninteresting. That is until he meets a troublesome couple, the supposedly charismatic Sato Kakuzo and his girlfriend, the alluring,Jito Joo. Clearly disturbed, they play games and place wagers where the loser has to physically harm himself. They attach to Oda, inducing him into  a wager after plying him with alcohol. After losing the game, he signs a detailed confession admitting culpability in the disappearances. Joo delivers the confession to the police and Oda is soon arrested, imprisoned, abused, tried and convicted. He is subsequently sentenced to death by hanging and executed, remaining silent thro

array(['Sorry for the disappointment'], dtype='<U28')

In [74]:
from sklearn.linear_model import LogisticRegression
clf_log = LogisticRegression()
clf_log.fit(train_x_vectors,train_y)

print(test_x[98])
clf_log.predict(test_x_vectors[98])

LOVE THE SERIES OF BOOKS WITH HARRY HOLE. THEY ARE REALLY EXCITING AND A LITTLE BIT DARK. HE IS A GREAT AUTHOR. DON'T MISS ANY OF HIS BOOKS.


array(['Thanks, It means alot'], dtype='<U28')

### Evaluation

In [90]:
print(clf_dec.score(test_x_vectors,test_y))
print(clf_log.score(test_x_vectors,test_y))
print(clf_svm.score(test_x_vectors,test_y))

0.6225961538461539
0.8052884615384616
0.8076923076923077


In [76]:
from sklearn.metrics import f1_score

# f1_score of svm model
print(f1_score(test_y,clf_svm.predict(test_x_vectors),average=None,labels=[Sentiment.Positive,Sentiment.Netural,Sentiment.Negative]))
# f1_score of decision tree .
print(f1_score(test_y,clf_dec.predict(test_x_vectors),average=None,labels=[Sentiment.Positive,Sentiment.Netural,Sentiment.Negative]))
# f1_score of Logistic reg
print(f1_score(test_y,clf_log.predict(test_x_vectors),average=None,labels=[Sentiment.Positive,Sentiment.Netural,Sentiment.Negative]))


[0.80582524 0.         0.80952381]
[0.62529833 0.         0.61985472]
[0.80291971 0.         0.80760095]


C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


In [83]:
test_set = ['This book is much super']
# test_set = ["this is the best book",'This is the worst than ever','I expected more but you have disappointed me','']
new_set = vectorizer.transform(test_set)
# print(vectorizer.get_feature_names())
# print(new_set[0])

clf_svm.predict(new_set)

array(['Thanks, It means alot'], dtype='<U28')

### Tuning model

In [88]:
from sklearn.model_selection import GridSearchCV

parameter = {'kernel':('linear','rbf'),'C':(1,4,8,16,32)}

svc = svm.SVC()
clf = GridSearchCV(svc,parameter,cv=5)

clf.fit(train_x_vectors,train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [91]:
clf.score(test_x_vectors,test_y)

0.8052884615384616

### Saving Model

In [94]:
import pickle
# here we can save the model. So we can call anytime
with open('./Machine Models/Review_sentiment.pkl','wb') as f:
    pickle.dump(clf,f)

### Load Model

In [95]:
with open('./Machine Models/Review_sentiment.pkl','rb') as f:
    load_clf = pickle.load(f)

In [96]:
print(test_x[67])
load_clf.predict(test_x_vectors[67])

For me this book was just two and a half stars. The characters took too long to get together with silly misunderstandings. When they finally got together she runs away from him with another misunderstanding. The end felt too rushed. I was annoyed with the second book when they were little together because of the tour. I thought they would be together more in the last book and got disappointed when that didn't happened.


array(['Sorry for the disappointment'], dtype='<U28')